In [2]:
import numpy as np

## List approach (so slow)

In [155]:
uncertainty_dict = {}
uncertainty_list = []
x_range = np.arange(-5,5.1,0.1)
y_range = np.arange(-5,5.1,0.1)
z_range = np.arange(0,5.1,0.1)

N = len(x_range)
for i in range(len(x_range)):
    for j in range(len(y_range)):
        for k in range(len(z_range)):
            element = (x_range[i], y_range[j], z_range[k])
            uncertainty_dict[element] = 1
            uncertainty_list.append([x_range[i], y_range[j], z_range[k]])
        
uncertainty_grids = np.array(uncertainty_list)
uncertainty_values = np.ones((uncertainty_grids.shape[0], ))

In [44]:
dist = 1e6
eps = 0.25
current_pos = [1.0, 2.0, 3.5]
closest_grid = None

for key, value in uncertainty_list.items():
    current_dist = np.sqrt((key[0]-current_pos[0])**2 + (key[1]-current_pos[1])**2 + (key[2]-current_pos[2])**2)
    if current_dist < dist:
        dist = current_dist
        closest_grid = key
        
    if current_dist <= eps:
        print ("that's right")
        break
        
closest_grid

that's right


(1.0, 2.0, 3.5)

## Array approach (much faster)

In [51]:
x_lim, y_lim, z_lim = 20, 20, 15
eps = 0.1
res = 1
X,Y,Z = np.mgrid[-x_lim:x_lim+eps:res, -y_lim:y_lim+eps:res, 0:z_lim+eps:res]
uncertainty_grids = np.vstack((X.flatten(), Y.flatten(), Z.flatten())).T
uncertainty_values = np.ones((uncertainty_grids.shape[0], ))
grid_visits = np.zeros((uncertainty_grids.shape[0], ))

print (uncertainty_grids.shape)

(26896, 3)


In [52]:
current_pos = [1.75, 2.0, 3.5]

differences = current_pos-uncertainty_grids
distances = np.sum(differences*differences,axis=1)
min_ind = np.argmin(distances)
print("Closest grid: ",uncertainty_grids[min_ind])
print("Uncertainty value: ", uncertainty_values[min_ind])

# Closest N grids
dist = 5.0
indices = np.where(distances<dist)[0]
print ("Closest N Grids: ", len(indices))

Closest grid:  [2. 2. 3.]
Uncertainty value:  1.0
Closest N Grids:  50


In [53]:
uncertainty_values[distances<dist] = 0.3

grid_visits[distances<dist] += 1

In [57]:
np.clip(np.exp(-grid_visits[distances<dist]), 1e-6, 1.0)

array([0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944,
       0.36787944, 0.36787944, 0.36787944, 0.36787944, 0.36787944])

In [73]:
W = np.sqrt(uncertainty_grids.shape[0])
K, S, P, D = 7, 2, 1, 1
K2, S, P2, D = 2, 2, 0, 1
print ("Width: ", W)
out1 = np.floor((W + 2*P - D*(K - 1) - 1) / S + 1)
print ("Conv2D out1: ", out1)

out2 = np.floor((out1 + 2*P2 - D*(K2 - 1) - 1) / S + 1)
print ("MaxPooling out2: ", out2)

out3 = np.floor((out2 + 2*P - D*(K - 1) - 1) / S + 1)
print ("Conv2D out3: ", out3)

out3 = np.floor((out2 + 2*P2 - D*(K2 - 1) - 1) / S + 1)
print ("MaxPooling out4: ", out4)

out4

Width:  164.0
Conv2D out1:  80.0
MaxPooling out2:  40.0
Conv2D out3:  18.0
MaxPooling out4:  6.0


6.0

In [58]:
b = 100
a = "agent_" + str(b) + ".pt"
a

'agent_100.pt'

In [6]:
np.reshape(uncertainty_values, (164,164))

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])